In [3]:
# !pip install spacy==3.5.4
# !python -m spacy download en_core_web_sm
# !pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.3 MB/s eta 0:00:0000:0100:01


In [4]:
import os
import re
import datetime
import requests
import pandas as pd
import spacy
import psutil
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
from rapidfuzz import process, fuzz
import string
import gc



nlp = spacy.load("en_core_web_sm")

# === Clean paragraph text ===
def clean_text(raw_text):
    text = raw_text.replace('\n', ' ')
    text = re.sub(r'-\s+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace('—', '-').replace('“', '"').replace('”', '"').replace("’", "'")
    text = re.sub(r'CHAPTER\s[\w\d\.]+.*?(?=\s)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^[A-Z\s\.,\'\-]{10,}$', '', text, flags=re.MULTILINE)
    return text

# === Extract cleaned sentences in batches ===
def get_text(url, explorer_name, explorer_id, buffer_size=100):
    url = url.strip()
    now = datetime.datetime.now()

    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Bad Status: {e}, for url: {url}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    all_sentences = []
    buffer = []

    for tag in soup.find_all('p'):
        content = tag.get_text(strip=True)
        if content:
            buffer.append(content)

        if len(buffer) >= buffer_size:
            chunk_text = ' '.join(buffer)
            buffer = []

            cleaned = clean_text(chunk_text)
            sentences = sent_tokenize(cleaned)
            all_sentences.extend(s for s in sentences if len(s.split()) >= 5)

            print(f"🧠 Memory Used: {psutil.Process(os.getpid()).memory_info().rss / 1e6:.2f} MB")

    if buffer:
        chunk_text = ' '.join(buffer)
        cleaned = clean_text(chunk_text)
        sentences = sent_tokenize(cleaned)
        all_sentences.extend(s for s in sentences if len(s.split()) >= 5)

    print(f"✅ Extracted {len(all_sentences)} sentences from {explorer_name} (ID: {explorer_id})")
    print(f"⏱️ Time taken: {datetime.datetime.now() - now}")

    return all_sentences

def match_sentence_to_paragraph_fast(sentence, paragraphs, threshold=90):
    """
    Efficiently match a sentence to the best paragraph using RapidFuzz.
    Returns best paragraph if similarity ≥ threshold, else empty string.
    """
    best_match = process.extractOne(
        sentence,
        paragraphs,
        scorer=fuzz.token_set_ratio
    )
    if best_match and best_match[1] >= threshold:
        return best_match[0]
    return ""

def clean_for_matching(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Chunk paragraphs into blocks
def chunk_paragraphs(paragraphs, block_size=100):
    for i in range(0, len(paragraphs), block_size):
        yield paragraphs[i:i + block_size]


# Hybrid match: exact first, fallback to fuzzy search on blocks
def match_sentence_to_paragraph_hybrid_optimized(
    sentence, paragraph_map, cleaned_paragraphs, original_paragraphs,
    threshold=90, block_size=100):
    
    # 1. Try exact match (original tokenized)
    exact_match = paragraph_map.get(sentence.strip())
    if exact_match:
        return exact_match

    # 2. Fuzzy match fallback
    cleaned_sentence = clean_for_matching(sentence)
    best_score = 0
    best_para = ""

    for block in chunk_paragraphs(cleaned_paragraphs, block_size):
        match = process.extractOne(cleaned_sentence, block, scorer=fuzz.token_set_ratio)
        if match and match[1] > best_score:
            best_score = match[1]
            best_para = match[0]

    if best_score >= threshold:
        idx = cleaned_paragraphs.index(best_para)
        return original_paragraphs[idx]

    return ""


# === Extract location entities ===
def extract_locations(text):
    doc = nlp(text)
    return {ent.text for ent in doc.ents if ent.label_ in {"GPE", "LOC", "FAC"}}
    

# === Main pipeline ===
def enrich_classified_using_book_name_and_url(diary_url_csv, classified_csv, output_path=None, buffer_size=100, batch_size=5):
    diary_url_df = pd.read_csv(diary_url_csv)
    df_classified = pd.read_csv(classified_csv)
    diary_url_df.columns = [col.lower() for col in diary_url_df.columns]
    df_classified.columns = [col.lower() for col in df_classified.columns]

    if 'sentence' not in df_classified.columns or 'book_name' not in df_classified.columns:
        raise ValueError("Classified CSV must contain 'sentence' and 'book_name' columns.")
    if not {'explorer', 'diary', 'id'}.issubset(diary_url_df.columns):
        raise ValueError("URL CSV must have 'explorer', 'diary', and 'id' columns.")

    all_results = []
    explorers = df_classified['book_name'].dropna().unique()

    for batch_start in range(0, len(explorers), batch_size):
        batch_explorers = explorers[batch_start:batch_start + batch_size]

        for explorer in batch_explorers:
            start_dt = datetime.datetime.now()
            print(f"starting at :{start_dt}")
            match = diary_url_df[diary_url_df['explorer'].str.strip().str.upper() == explorer.strip().upper()]
            if match.empty:
                print(f"❌ No URL found for book_name: {explorer}")
                continue

            url = match.iloc[0]['diary']
            explorer_id = match.iloc[0]['id']
            explorer_name = match.iloc[0]['explorer']
            print(f"\n🔗 Processing {explorer_name} ({explorer_id}) from: {url}")

            parsed_sentences = get_text(url, explorer_name, explorer_id, buffer_size=buffer_size)
            if not parsed_sentences:
                continue

            # Build sentence-to-paragraph map
            paragraph_map = {}
            for para in parsed_sentences:
                for sent in sent_tokenize(para):
                    if len(sent.split()) >= 5:
                        paragraph_map[sent.strip()] = para

            cleaned_paragraphs = [clean_for_matching(p) for p in parsed_sentences]

            # Filter classified rows for this explorer
            classified_subset = df_classified[df_classified['book_name'].str.strip().str.upper() == explorer.strip().upper()]

            for _, row in classified_subset.iterrows():
                sentence = row['sentence'].strip()

                paragraph = match_sentence_to_paragraph_hybrid_optimized(
                    sentence=sentence,
                    paragraph_map=paragraph_map,
                    cleaned_paragraphs=cleaned_paragraphs,
                    original_paragraphs=parsed_sentences,
                    threshold=90,
                    block_size=100
                )

                loc_sent = extract_locations(sentence)
                loc_para = extract_locations(paragraph)
                para_only = loc_para - loc_sent

                enriched = {
                    **row.to_dict(),
                    "paragraph": paragraph,
                    "locations_in_sentence": list(loc_sent),
                    "locations_in_paragraph_only": list(para_only),
                    "all_locations": list(loc_sent.union(loc_para)),
                    "num_locations": len(loc_sent.union(loc_para)),
                    "explorer_id": explorer_id,
                    "explorer_name": explorer_name,
                    "source_url": url
                }

                all_results.append(enriched)

            # === CLEAN UP after each explorer ===
            print(f"🧹 Cleaning up memory for {explorer_name} (ID: {explorer_id})")
            del parsed_sentences, paragraph_map, cleaned_paragraphs, classified_subset
            gc.collect()
            print(f"finishing at :{datetime.datetime.now()}, time took: {datetime.datetime.now() - start_dt}")

    df_out = pd.DataFrame(all_results)

    if output_path:
        df_out.to_csv(output_path, index=False)
        print(f"✅ Saved enriched output to: {output_path}")

    return df_out

In [ ]:
df = enrich_classified_using_book_name_and_url(
    diary_url_csv="/cluster/home/hlee37/git/gutenberg/phase2/DataForJohn.csv",
    classified_csv="/cluster/home/hlee37/git/gutenberg/phase4/output/filtered_sentences_0.csv",
    output_path="all_with_locations_2.csv",
    buffer_size=100
)

starting at :2025-06-26 17:56:38.985682

🔗 Processing Edward John Eyre (10) from: https://gutenberg.net.au/ebooks/e00048.html 
🧠 Memory Used: 3006.79 MB
🧠 Memory Used: 3318.54 MB
🧠 Memory Used: 3471.15 MB
🧠 Memory Used: 3553.23 MB
🧠 Memory Used: 3654.09 MB
🧠 Memory Used: 3734.97 MB
🧠 Memory Used: 3811.93 MB
🧠 Memory Used: 3869.72 MB
🧠 Memory Used: 3922.41 MB
🧠 Memory Used: 3993.79 MB
🧠 Memory Used: 4059.10 MB
🧠 Memory Used: 4102.46 MB
🧠 Memory Used: 4084.88 MB
🧠 Memory Used: 4181.56 MB
🧠 Memory Used: 4163.87 MB
🧠 Memory Used: 4239.10 MB
🧠 Memory Used: 4226.20 MB
🧠 Memory Used: 4215.98 MB
🧠 Memory Used: 4328.09 MB
🧠 Memory Used: 4278.16 MB
🧠 Memory Used: 4287.34 MB
🧠 Memory Used: 4291.52 MB
✅ Extracted 5521484 sentences from Edward John Eyre (ID: 10)
⏱️ Time taken: 0:02:29.868631
🧹 Cleaning up memory for Edward John Eyre (ID: 10)
finishing at :2025-06-26 18:00:52.152793, time took: 0:04:13.167131
starting at :2025-06-26 18:00:52.153028

🔗 Processing Ludwig Leichhardt (11) from: https://

In [ ]:
df.to_csv('final.csv')

In [1]:
import pandas as pd
import spacy

# Load SpaCy NER
nlp = spacy.load("en_core_web_sm")

# === Location extractor ===
def extract_locations(text):
    doc = nlp(text)
    return {ent.text for ent in doc.ents if ent.label_ in {"GPE", "LOC", "FAC"}}

# === Main function ===
def enrich_classified_with_locations(classified_csv, output_path=None):
    df = pd.read_csv(classified_csv)
    df.columns = [col.lower() for col in df.columns]

    if 'sentence' not in df.columns:
        raise ValueError("CSV must contain a 'sentence' column.")

    results = []
    for _, row in df.iterrows():
        sentence = row['sentence'].strip()
        locations = extract_locations(sentence)

        enriched = {
            **row.to_dict(),
            "locations_in_sentence": list(locations),
            "num_locations": len(locations)
        }

        results.append(enriched)

    df_out = pd.DataFrame(results)

    if output_path:
        df_out.to_csv(output_path, index=False)
        print(f"✅ Saved output to {output_path}")

    return df_out


/cluster/home/hlee37/condaenv/torch_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
enrich_classified_with_locations("/cluster/home/hlee37/git/gutenberg/phase4/output/filtered_sentences_0.csv", output_path="locations_output.csv")

✅ Saved output to locations_output.csv


,sentence,pos_prob,neg_prob,book_name,locations_in_sentence,num_locations
0,In offering to the public an account of Expedi...,0.905523,0.094477,Edward John Eyre,"[Australia, England, South Australia, the Murr...",4
1,"33 degrees 14 minutes S, is a practicable pass...",0.000112,0.999888,Edward John Eyre,[],0
2,"By digging in the gravelly bed of the channel,...",0.295373,0.704627,Edward John Eyre,[],0
3,"In descending, I followed a little rocky gully...",0.999435,0.000565,Edward John Eyre,[],0
4,We had been so long living upon nothing but th...,0.999568,0.000432,Edward John Eyre,[],0
...,...,...,...,...,...,...
3110,The combat now became general--spears flew in ...,0.000052,0.999948,Sir Thomas Mitchell,[],0
3111,It was now seen that further resistance would ...,0.000068,0.999932,Sir Thomas Mitchell,[],0
3112,When attempting to make their escape a line wa...,0.000060,0.999940,Sir Thomas Mitchell,[],0
3113,During their stay every precaution was taken b...,0.893572,0.106428,Sir Thomas Mitchell,[],0
